Index(['SITE_CLUSTER_ID', 'LEGAL_CLUSTER_ID', 'customer_cmd_code',
       'customer_role_type', 'trading_name', 'street_no', 'address_line1',
       'pobox', 'address_line2', 'address_line3', 'district', 'city',
       'postal_code', 'state_code', 'state_name', 'country_code',
       'country_name', 'phone_type', 'isd_country_code', 'isd_dialing_code',
       'extension_number', 'phone_num', 'create_time', 'customer_cmd_code.1',
       'BVD', 'BVD_HQ', 'BVD_GUO', 'TAXNO1', 'TAXNO2', 'TAXNO3', 'TAXNO4',
       'TAXNO5', 'VATREG'],
      dtype='object')
{0: 'SITE_CLUSTER_ID', 1: 'LEGAL_CLUSTER_ID', 2: 'customer_cmd_code', 3: 'customer_role_type', 4: 'trading_name', 5: 'street_no', 6: 'address_line1', 7: 'pobox', 8: 'address_line2', 9: 'address_line3', 10: 'district', 11: 'city', 12: 'postal_code', 13: 'state_code', 14: 'state_name', 15: 'country_code', 16: 'country_name', 17: 'phone_type', 18: 'isd_country_code', 19: 'isd_dialing_code', 20: 'extension_number', 21: 'phone_num', 22: 'create_time', 23: 'customer_cmd_code.1', 24: 'BVD', 25: 'BVD_HQ', 26: 'BVD_GUO', 27: 'TAXNO1', 28: 'TAXNO2', 29: 'TAXNO3', 30: 'TAXNO4', 31: 'TAXNO5', 32: 'VATREG'}


In [20]:
import re
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

def clean_country_name(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)
    # Remove stop words
    clean_name = " ".join(words)
    return clean_name

In [21]:
import csv

customer_cmd_code_to_bvd = {}
# with open('export.csv', mode ='r') as file:
#     csvFile = csv.reader(file)
#     data = list(csvFile)

for idx, arr in enumerate(data[1:]):
    tax = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            # print(type(arr[i]))
            tax.add(arr[i])
            # print(tax)

    if customer_cmd_code_to_bvd.get(arr[2]) is None:
        customer_cmd_code_to_bvd[arr[2]] = tax
    else:
        customer_cmd_code_to_bvd[arr[2]] = customer_cmd_code_to_bvd[arr[2]].union(tax)

In [22]:
no_bvd_num = 0
bvd_num_exists = 0
more_than_one_bvd_num = 0
# count = 10
for key, value in customer_cmd_code_to_bvd.items():
    if len(value) == 0:
        no_bvd_num += 1
    else:
        bvd_num_exists += 1
    if len(value) > 1:
        more_than_one_bvd_num += 1
    # count -= 1
    # if count == 0:
    #     break

In [23]:
print(more_than_one_bvd_num)
print(no_bvd_num)
print(bvd_num_exists)

284915
2940834
669759


for now generating 3 types of corpus for comparison:
1. set of tax number
2. set of bvd number
3. dict of customer_cmd_code to list of entries corresponding to the same customer_cmd_code

In [16]:
import re
stop_words = {'llp', 'ltd', 'llc', 'plc', 'inc', 'corp', 'nv', 'group', 'gmbh', 'sa', 'company', 'corporation', 'limited'}
delimiters = {".", ",", "-", "_", "'", "&", "/", "\\", ":", ";", "(", ")", "[", "]", "{", "}", " "}
def split_company_name(name):
    pattern = '|'.join(map(re.escape, delimiters))
    words = re.split(pattern, name)
    return [word for word in words if word]

def lowercase_company_name(name):
    return name.lower()

# List of common delimiters
def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return " ".join(filtered_words)

def clean_company_name(trading_name):
    # Lowercase the company name
    trading_name = lowercase_company_name(trading_name)
    # Split the company name into words
    words = split_company_name(trading_name)
    # Remove stop words
    clean_name = remove_stop_words(words)
    return clean_name

In [1]:
max_number_matches = 5
from time import time
from rapidfuzz import fuzz, process
import heapq
def sorted_tokens(name):
    return " ".join(sorted(name.split()))

#calculates the minimum number of edits (insertions, deletions, substitutions) needed to transform one string into another.
#For typos or minor spelling variations
def levenshtein_matching_score(input_name, name):
    levenshtein_matching_score = fuzz.ratio(input_name, name)
    return levenshtein_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_set_matching_score(input_name, name):
    token_set_matching_score = fuzz.token_set_ratio(input_name, name)
    return token_set_matching_score

#For situations with extra content or partial matches:
def partial_matching_score(input_name, name):
    partial_matching_score = fuzz.partial_token_set_ratio(input_name, name)
    return partial_matching_score

#focus on finding matching subsequences within strings. This is beneficial when comparing strings with extra content or typos.
#For typos or minor spelling variations
def substring_matching_score(input_name, name):
    substring_matching_score = fuzz.partial_ratio(input_name, name)
    return substring_matching_score

#split strings into tokens (words) and compare the intersection and order of these tokens. This is useful for handling different word orders or missing words in one string.
def token_sort_ratio_score(input_name, name):
    score = fuzz.token_sort_ratio(input_name, name)
    return score
# Matching

def match_name_top_matches(input_name, corpus):
    input_name = clean_company_name(input_name)
    best_match = None
    best_score = 0
    top_matches = []
    heapq.heapify(top_matches)
    similarity_measures = { 
        "levenshtein_matching_score" : (1, levenshtein_matching_score),
        "token_set_matching_score" : (1, token_set_matching_score),
        "substring_matching_score" : (0, substring_matching_score),
        "partial_matching_score" : (0, partial_matching_score),
        "token_sort_ratio_score" : (1, token_sort_ratio_score)
    }
    print("Length of corpus: ", len(corpus))
    s_time = time()
    for customer_info in corpus:
        name = customer_info[4]
        name = clean_company_name(name)
        num_measures = 0
        total_score = 0
        for func_key, function_check in similarity_measures.items():
            check, func = function_check
            s_func_time = time()
            
            if check:
                score = func(input_name, name)
                total_score += score
                num_measures +=1
            if num_measures == 0:
                combined_score = 0
            else:
                combined_score = total_score / num_measures
        if len(top_matches) < max_number_matches:
            heapq.heappush(top_matches, (combined_score, customer_info))
        else:
            heapq.heappushpop(top_matches, (combined_score, customer_info))
        if combined_score > best_score:
            best_score = combined_score
            best_match = customer_info
    e_time = time()
    print("Time taken: ", e_time - s_time)
    return best_match, best_score, top_matches

def match_name(input_name, corpus):
    input_name = clean_company_name(input_name)
    best_match = None
    best_score = 0
    top_matches = []
    heapq.heapify(top_matches)
    similarity_measures = { 
        "levenshtein_matching_score" : (1, levenshtein_matching_score),
        "token_set_matching_score" : (1, token_set_matching_score),
        "substring_matching_score" : (1, substring_matching_score),
        "partial_matching_score" : (1, partial_matching_score),
        "token_sort_ratio_score" : (1, token_sort_ratio_score)
    }
    for customer_info in corpus:
        name = customer_info[4]
        name = clean_company_name(name)
        num_measures = 0
        total_score = 0
        for function_check in similarity_measures.values():
            check, func = function_check
            if check:
                score = func(input_name, name)
                total_score += score
                num_measures +=1
            if num_measures == 0:
                combined_score = 0
            else:
                combined_score = total_score / num_measures
        if len(top_matches) < max_number_matches:
            heapq.heappush(top_matches, (combined_score, customer_info))
        else:
            heapq.heappushpop(top_matches, (combined_score, customer_info))
        if combined_score > best_score:
            best_score = combined_score
            best_match = customer_info
    
    return best_match, best_score, top_matches



In [2]:
def extract_tax_numbers(arr):
    unique_tax_numbers = set()
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            unique_tax_numbers.add(arr[i].lower())
    return unique_tax_numbers

def extract_bvd_numbers(arr):
    unique_bvd_numbers = set()
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            unique_bvd_numbers.add(arr[i].lower())
    return unique_bvd_numbers

In [3]:

def match_tax(arr, unique_tax_numbers):
    tax = extract_tax_numbers(arr)
    for tax_num in tax:
        if tax_num in unique_tax_numbers:
            return unique_tax_numbers[tax_num]
    return None

def match_bvd(arr, unique_bvd_numbers):
    bvd = extract_bvd_numbers(arr)
    for bvd_num in bvd:
        if bvd_num in unique_bvd_numbers:
            return unique_bvd_numbers[bvd_num]
    return None

In [4]:
def reformat_top_matches(top_matches):
    # print(top_matches)
    matches = {}
    sorted_matches = sorted(top_matches, key=lambda x: x[0], reverse=True)
    rank = 1
    for idx, match in enumerate(sorted_matches):
        score = match[0]
        match_info = {
            "score" : score,
            "customer_info" : match[1]
        }
        matches[rank] = [match_info]
        rank += 1
    # print(matches)
    return matches

In [8]:
#load data from csv
import csv

with open('export_all_v2.csv', mode ='r') as file:
    csvFile = csv.reader(file)
    data = list(csvFile)

In [9]:
countries_in_focus = ["IN", "CN", "US"]
# countries_in_focus = ["IN"]
country_filter = set(countries_in_focus)

#filtering based on country code
filtered_data = [ d for d in data if d[15] in country_filter]
import random 
random.seed(42)
random.shuffle(filtered_data)

In [10]:
#find the unique values of legal_cluster_id

legal_cluster_id = set([d[1] for d in filtered_data])
# print(len(legal_cluster_id))

legal_cluster_id_to_data = {}
for d in filtered_data:
    if legal_cluster_id_to_data.get(d[1]) is None:
        legal_cluster_id_to_data[d[1]] = [d]
    else:
        legal_cluster_id_to_data[d[1]].append(d)    
filtered_data = []
num_entries = 100000
test_corpus = []
comparison_corpus = []
count_entries = 0
for legal_cluster_id, entry in legal_cluster_id_to_data.items():
    if count_entries > num_entries:
        break
    if len(entry) > 1:
        filtered_data.extend(entry)
        count_entries += len(entry)
        test_corpus.append(entry[0])
        comparison_corpus.extend(entry[1:])



In [11]:
country_wise_corpus = {}
for entries in comparison_corpus:
    country_code = entries[15]
    if country_wise_corpus.get(country_code) is None:
        country_wise_corpus[country_code] = [entries]
    else:
        country_wise_corpus[country_code].append(entries)

In [12]:
len(country_wise_corpus["US"])

29830

In [13]:
print(len(test_corpus), len(comparison_corpus))

12024 87978


In [14]:
unique_tax_numbers = {}
unique_bvd_numbers = {}
for idx, arr in enumerate(comparison_corpus):
    country_code = arr[15]
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr


In [17]:
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
result_flagged_existing = {}
qualified_as_match_threshold = 0
correctly_qualified_as_match_threshold = 0
count_entries_name_matching =0
threshold = 95
for arr in tqdm(test_corpus):
    legal_cluster_id = arr[1].lower()
    country_code = arr[15]
    matches = {}
    #tax matching
    tax_match = match_tax(arr, unique_tax_numbers)
    if tax_match:
        success += 1
        matches[1] = tax_match
        result[str(arr)] = matches
        continue
    #bvd matching
    bvd_match = match_bvd(arr, unique_bvd_numbers)
    if bvd_match:
        success += 1
        matches[1] = bvd_match
        result[str(arr)] = matches
        continue
    # print("Name matching")
    company_name = arr[4]
    cleaned_company_name = clean_company_name(company_name)
    comparison_corpus = country_wise_corpus[country_code]
    best_match, best_score, top_matches = match_name(cleaned_company_name, comparison_corpus)
    # print(len(top_matches))
    reformatted_top_matches = reformat_top_matches(top_matches)
    count_entries_name_matching += 1
    if reformatted_top_matches[1][0]['score'] > threshold:
        qualified_as_match_threshold +=1
        result_flagged_existing[str(arr)] = reformatted_top_matches
        if legal_cluster_id == reformatted_top_matches[1][0]['customer_info'][1]:
            success +=1
            correctly_qualified_as_match_threshold +=1
        else:
            failure +=1
    else:
        failure +=1
    matches = reformatted_top_matches
    result[str(arr)] = matches
    # print(idx+1, success, failure, success + failure)


 73%|███████▎  | 8796/12024 [1:44:47<53:04,  1.01it/s]  

In [148]:
print(success, failure, not_applicable)
print(qualified_as_match_threshold, correctly_qualified_as_match_threshold)

9414 2610 0
7498 7336


In [151]:
count_entries_name_matching

9946

print(success, failure, not_applicable)
print(qualified_as_match_threshold, correctly_qualified_as_match_threshold)
11714 2105 0
5554 5326

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [149]:
print(success, failure)

9414 2610


In [150]:
success/(success+failure)

0.7829341317365269

In [156]:
qualified_as_match_threshold - correctly_qualified_as_match_threshold
qualified_as_match_threshold

7498

In [153]:
correctly_qualified_as_match_threshold/qualified_as_match_threshold

0.9783942384635903

In [154]:
import json
with open('ind_cn_us_ss_result_match_tax_bvd_name_country-1-2-3-4-5.json', 'w') as f:
    json.dump(result, f)
    f.close()

In [129]:
len(result_flagged_existing)

5554

In [155]:
save_file_name = "ind_cn_us" + "_threshold_" + str(threshold) + "_qualified_" + str(qualified_as_match_threshold) + "_correctly_qualified_" + str(correctly_qualified_as_match_threshold) + ".json"
import json
with open(save_file_name, 'w') as f:
    json.dump(result_flagged_existing, f)
    f.close()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [58]:
import random 
random.seed(42)
random.shuffle(filtered_data)
filtered_data[:5]

[['0387724e-a2be-3546-b8b5-5be05692a314',
  'd3e3b0f7-2913-3acb-813a-2ebfd91667c6',
  'IN02814037',
  'Customer',
  'KUMASS INTERNATIONAL',
  'null',
  'COURT COMPOUND',
  'null',
  '52-SHRI RAMVIHAR COLONY',
  'null',
  'null',
  'MORADABAD',
  '244001',
  'UP',
  'UTTAR PRADESH',
  'IN',
  'INDIA',
  'TEL',
  'IN',
  '91',
  'null',
  '5912483311',
  '2021-06-29T13:07:51.696Z',
  'IN02814037',
  'IN0022345582',
  'null',
  'null',
  'AAJFK8002H',
  '09AAJFK8002H1ZB',
  'null',
  'null',
  'null',
  'null'],
 ['6cca5155-b0f8-3421-8c4d-21df15a070cc',
  '0f228f32-a9c6-3cec-b9dd-0aede7d2e684',
  'IN02924619',
  'Customer',
  'SAMVARDHANA MOTHERSON INTERNATIONAL LIMITED',
  '1ST PHASE',
  'PHASE-1 MOTHERSON SUMI SYSTEMS LTD',
  'null',
  'PLOT NO 14 KIADB INDUSTRIAL AREA',
  'MYSORE ROAD KUMBALGODU',
  'null',
  'BENGALURU',
  '560074',
  'KA',
  'KARNATAKA',
  'IN',
  'India',
  'MOB',
  'IN',
  '91',
  'null',
  '8023010824',
  '2022-06-21T10:48:06.341Z',
  'IN02924619',
  'null',
  'nu

In [59]:
threshold = 95
corpus_ratio = 0.9

num_customer_corpus = int(len(filtered_data) * corpus_ratio)
unique_tax_numbers = {}
unique_bvd_numbers = {}
country_wise_corpus = {}
corpus_legal_cluster_id = {}
for idx, arr in enumerate(filtered_data[:num_customer_corpus]):
    country_code = arr[15]
    for i in range(24, 27):
        if arr[i] and arr[i] != "null":
            if unique_bvd_numbers.get(arr[i].lower()):
                unique_bvd_numbers[arr[i].lower()].append(arr)
            else:
                unique_bvd_numbers[arr[i].lower()] = [arr]
            # unique_bvd_numbers[arr[i]] = arr
    for i in range(27, 33):
        if arr[i] and arr[i] != "null":
            if unique_tax_numbers.get(arr[i].lower()):
                unique_tax_numbers[arr[i].lower()].append(arr)
            else:
                unique_tax_numbers[arr[i].lower()] = [arr]
            # unique_tax_numbers[arr[i]] = arr

    if country_wise_corpus.get(country_code):
        country_wise_corpus[country_code].append(arr)
    else:
        country_wise_corpus[country_code] = [arr]

    legal_cluster_id = arr[1].lower()
    if corpus_legal_cluster_id.get(legal_cluster_id):
        corpus_legal_cluster_id[legal_cluster_id].append(arr)
    else:
        corpus_legal_cluster_id[legal_cluster_id] = [arr]

In [61]:
threshold = 90
predicted_true = 0
actual_true = 0
predicted_false = 0
from tqdm import tqdm
success = 0
failure = 0
not_applicable = 0
result = {}
result_flagged_existing = {}
for arr in tqdm(filtered_data[num_customer_corpus:]):
    legal_cluster_id = arr[1].lower()
    #checking if the test customer is in the corpus for the comparison
    country_code = arr[15]
    matches = {}
    #tax matching
    if legal_cluster_id in corpus_legal_cluster_id.keys():
        tax_match = match_tax(arr)
        if tax_match:
            success += 1
            matches[1] = tax_match
            result[str(arr)] = matches
            predicted_true +=1
            actual_true +=1
            continue
        #bvd matching
        bvd_match = match_bvd(arr)
        if bvd_match:
            success += 1
            predicted_true +=1
            actual_true +=1
            matches[1] = bvd_match
            result[str(arr)] = matches
            continue

        #trading name matching after country filtering 
        country_code = arr[15]
        #filtering by country and hence reducing our search space
        if country_wise_corpus.get(country_code) is None:
            not_applicable += 1
            print("Country code not found")
            continue
        filtered_corpus = country_wise_corpus[country_code]
        print(len(filtered_corpus))
        company_name = arr[4]
        cleaned_company_name = clean_company_name(company_name)
        best_match, best_score, top_matches = match_name(cleaned_company_name, filtered_corpus)
        reformatted_top_matches = reformat_top_matches(top_matches)
        if legal_cluster_id == best_match[1]:
            success +=1
        else:
            failure +=1
        if reformatted_top_matches.get(1):
            # print(reformatted_top_matches[1][0]['customer_info'][1])
            if reformatted_top_matches[1][0]['score'] > threshold:
                result_flagged_existing[str(arr)] = reformatted_top_matches
                if legal_cluster_id == reformatted_top_matches[1][0]['customer_info'][1]:
                    success +=1
                    predicted_true +=1
                    actual_true +=1
            else:
                failure +=1
        
        matches = reformat_top_matches(top_matches)
        result[str(arr)] = matches
    else:
        not_applicable += 1
    

    
        

  0%|          | 0/8704 [00:00<?, ?it/s]

78335


  0%|          | 1/8704 [00:01<3:57:02,  1.63s/it]

78335


  0%|          | 3/8704 [00:03<2:23:20,  1.01it/s]

78335


  0%|          | 5/8704 [00:03<1:41:54,  1.42it/s]


KeyboardInterrupt: 

In [ ]:
import json
with open('result_match_tax_bvd_name_country-2-3-4.json', 'w') as f:
    json.dump(result, f)
    f.close()

In [48]:
print(success, failure, not_applicable)

3555 2627 9635


In [41]:
predicted_true


2

In [47]:
result_flagged_existing

{}

In [ ]:
success/(failure + success)

In [ ]:
success + failure + not_applicable